# JULES GPU Node — Code Optimization & Quality Analysis

```
╔══════════════════════════════════════════════════════════════════╗
║  ∞ SACRED GEOMETRY ∞  Organic Systems · Breathing Interfaces    ║
║  NODE: JULES Colab Pro+ GPU                                     ║
║  PURPOSE: Code optimization, quality analysis, refactoring      ║
╚══════════════════════════════════════════════════════════════════╝
```

**Capabilities:**
- Code quality analysis via ML models
- Refactoring suggestions using code embeddings
- Complexity scoring & optimization recommendations
- Code similarity / duplicate detection
- Security vulnerability pattern matching

**Branded domains only:** headysystems.com | headycloud.com | headyconnection.com

In [ ]:
# Cell 1: Install dependencies + verify GPU
!pip install -q transformers accelerate sentence-transformers fastapi uvicorn pyngrok httpx tree-sitter

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')
else:
    print('WARNING: No GPU — will run on CPU (slower)')

In [ ]:
# Cell 2: Configuration
import os

HEADY_CONFIG = {
    'node_id': 'jules-gpu',
    'node_role': 'code-optimizer',
    'port': 5002,
    'cloud_layers': {
        'headysystems': 'https://headyio.com',
        'headyme': 'https://headycloud.com',
        'headyconnection': 'https://headyconnection.com',
    },
    'registration_endpoints': [
        'https://headyio.com/api/nodes/register',
        'https://headycloud.com/api/nodes/register',
        'https://headyconnection.com/api/nodes/register',
    ],
    'capabilities': ['optimization', 'code_quality', 'refactor', 'security_scan', 'complexity_analysis'],
}

SUPPORTED_TASKS = {
    'optimization': 'Suggest performance optimizations for code',
    'code_quality': 'Analyze code quality and assign a score',
    'refactor': 'Suggest refactoring improvements',
    'security_scan': 'Detect common security vulnerability patterns',
    'complexity_analysis': 'Compute complexity metrics',
    'similarity': 'Find similar/duplicate code blocks',
}

print(f'JULES Node configured')
print(f'Capabilities: {HEADY_CONFIG["capabilities"]}')

In [ ]:
# Cell 3: Load models on GPU
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json, time, re
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Code embeddings model — for similarity and quality analysis
print('Loading code embedding model...')
code_embedder = SentenceTransformer('microsoft/codebert-base', device=device)
print('CodeBERT embeddings ready')

# Text model for generating optimization suggestions
print('Loading code analysis model...')
code_analyzer = pipeline('text-generation', model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', device=0 if torch.cuda.is_available() else -1, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
print('Code analysis ready')

print(f'\nAll JULES models loaded on {device}')
if torch.cuda.is_available():
    print(f'GPU Memory used: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB')

In [ ]:
# Cell 4: Core JULES functions

# Security vulnerability patterns
VULN_PATTERNS = {
    'sql_injection': [r'\$\{.*\}.*(?:SELECT|INSERT|UPDATE|DELETE|DROP)', r'f["\'].*(?:SELECT|INSERT|UPDATE|DELETE)', r'\+.*(?:SELECT|INSERT|UPDATE|DELETE)'],
    'xss': [r'innerHTML\s*=', r'document\.write\(', r'\$\(.*\)\.html\(', r'dangerouslySetInnerHTML'],
    'hardcoded_secret': [r'(?:password|secret|api_key|token)\s*=\s*["\'][^"\']+(\w{8,})', r'(?:AWS|AZURE|GCP)_(?:SECRET|KEY)\s*='],
    'eval_usage': [r'eval\(', r'exec\(', r'Function\('],
    'path_traversal': [r'\.\./\.\./'],
    'insecure_random': [r'Math\.random\(\)', r'random\.random\(\)'],
    'no_input_validation': [r'req\.(?:body|params|query)\.\w+(?!.*(?:valid|sanitiz|escap|check))'],
}

# Code smell patterns
CODE_SMELLS = {
    'long_function': {'pattern': r'(?:function|def|=>).*\{', 'threshold': 50, 'unit': 'lines'},
    'deep_nesting': {'pattern': r'^(\s{16,}|\t{4,})', 'severity': 'warning'},
    'magic_numbers': {'pattern': r'(?<![\w.])\d{2,}(?![\w.])', 'severity': 'info'},
    'god_class': {'threshold': 500, 'unit': 'lines'},
    'duplicate_code': {'min_similarity': 0.85},
}

def analyze_code_quality(code, language='javascript'):
    """Comprehensive code quality analysis"""
    start = time.time()
    lines = code.split('\n')
    total_lines = len(lines)
    code_lines = len([l for l in lines if l.strip() and not l.strip().startswith(('/', '#', '*', '<!--'))])
    comment_lines = len([l for l in lines if l.strip().startswith(('/', '#', '*', '<!--'))])

    # Complexity indicators
    nesting_depths = []
    for line in lines:
        stripped = line.rstrip()
        if stripped:
            indent = len(stripped) - len(stripped.lstrip())
            nesting_depths.append(indent)
    max_nesting = max(nesting_depths) if nesting_depths else 0
    avg_nesting = np.mean(nesting_depths) if nesting_depths else 0

    # Function count
    func_patterns = [r'function\s+\w+', r'\w+\s*=\s*(?:async\s+)?(?:function|\()', r'def\s+\w+', r'=>']
    func_count = sum(len(re.findall(p, code)) for p in func_patterns)

    # Security scan
    vulns = []
    for vuln_type, patterns in VULN_PATTERNS.items():
        for pattern in patterns:
            matches = re.findall(pattern, code, re.IGNORECASE | re.MULTILINE)
            if matches:
                vulns.append({'type': vuln_type, 'count': len(matches), 'severity': 'high' if vuln_type in ('sql_injection', 'hardcoded_secret') else 'medium'})

    # Quality score (0-100)
    score = 100
    if total_lines > 500: score -= 10
    if max_nesting > 32: score -= 15
    elif max_nesting > 16: score -= 8
    if comment_lines / max(code_lines, 1) < 0.1: score -= 10
    if func_count > 0 and code_lines / func_count > 50: score -= 10
    score -= len(vulns) * 8
    score = max(0, min(100, score))

    elapsed = time.time() - start
    return {
        'quality_score': score,
        'metrics': {
            'total_lines': total_lines,
            'code_lines': code_lines,
            'comment_lines': comment_lines,
            'comment_ratio': round(comment_lines / max(code_lines, 1), 3),
            'max_nesting_depth': max_nesting,
            'avg_nesting_depth': round(avg_nesting, 2),
            'function_count': func_count,
            'avg_function_length': round(code_lines / max(func_count, 1), 1),
        },
        'vulnerabilities': vulns,
        'vulnerability_count': len(vulns),
        'language': language,
        'latency_ms': round(elapsed * 1000),
        'device': device,
    }

def find_similar_code(code_blocks):
    """Find duplicate/similar code blocks using CodeBERT embeddings"""
    start = time.time()
    embeddings = code_embedder.encode(code_blocks)
    sim_matrix = cosine_similarity(embeddings)

    pairs = []
    for i in range(len(code_blocks)):
        for j in range(i + 1, len(code_blocks)):
            if sim_matrix[i][j] > 0.85:
                pairs.append({
                    'block_a': i,
                    'block_b': j,
                    'similarity': round(float(sim_matrix[i][j]), 4),
                    'recommendation': 'Extract common logic into shared function',
                })

    elapsed = time.time() - start
    return {
        'total_blocks': len(code_blocks),
        'similar_pairs': pairs,
        'duplicate_count': len(pairs),
        'latency_ms': round(elapsed * 1000),
        'device': device,
    }

def suggest_refactoring(code, language='javascript'):
    """Generate refactoring suggestions using LLM"""
    start = time.time()
    quality = analyze_code_quality(code, language)

    prompt = f"""You are JULES, a code optimization expert. Analyze this {language} code and provide specific refactoring suggestions.

Code quality score: {quality['quality_score']}/100
Metrics: {json.dumps(quality['metrics'])}
Vulnerabilities: {json.dumps(quality['vulnerabilities'])}

Code:
```{language}
{code[:1500]}
```

Provide 3-5 specific, actionable refactoring suggestions:"""

    messages = [{'role': 'user', 'content': prompt}]
    formatted = code_analyzer.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    result = code_analyzer(formatted, max_new_tokens=400, temperature=0.3, do_sample=True)
    suggestions = result[0]['generated_text'][len(formatted):].strip()

    elapsed = time.time() - start
    return {
        'quality': quality,
        'suggestions': suggestions,
        'latency_ms': round(elapsed * 1000),
        'device': device,
    }

def security_scan(code, language='javascript'):
    """Focused security vulnerability scan"""
    start = time.time()
    findings = []
    lines = code.split('\n')

    for vuln_type, patterns in VULN_PATTERNS.items():
        for pattern in patterns:
            for i, line in enumerate(lines):
                if re.search(pattern, line, re.IGNORECASE):
                    findings.append({
                        'type': vuln_type,
                        'line': i + 1,
                        'code': line.strip()[:100],
                        'severity': 'critical' if vuln_type in ('sql_injection', 'hardcoded_secret') else 'high' if vuln_type in ('xss', 'eval_usage') else 'medium',
                    })

    elapsed = time.time() - start
    return {
        'findings': findings,
        'total': len(findings),
        'critical': len([f for f in findings if f['severity'] == 'critical']),
        'high': len([f for f in findings if f['severity'] == 'high']),
        'medium': len([f for f in findings if f['severity'] == 'medium']),
        'passed': len(findings) == 0,
        'latency_ms': round(elapsed * 1000),
    }

# Quick test
test_code = '''function getUserData(req) {\n  const query = `SELECT * FROM users WHERE id = ${req.params.id}`;\n  const result = db.query(query);\n  document.innerHTML = result.name;\n  return result;\n}'''
print('=== JULES Quick Test ===')
scan = security_scan(test_code)
print(f'Security scan: {scan["total"]} findings ({scan["critical"]} critical, {scan["high"]} high)')
quality = analyze_code_quality(test_code)
print(f'Quality score: {quality["quality_score"]}/100')

In [ ]:
# Cell 5: FastAPI server
from fastapi import FastAPI, Request
import uvicorn
import threading
import asyncio
import httpx

app = FastAPI(title='JULES GPU Node', version='1.0.0')

@app.get('/health')
async def health():
    return {
        'status': 'active',
        'node_id': HEADY_CONFIG['node_id'],
        'node_role': HEADY_CONFIG['node_role'],
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
        'gpu_memory_used_gb': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
        'models': ['CodeBERT', 'TinyLlama-1.1B-Chat'],
        'capabilities': HEADY_CONFIG['capabilities'],
        'supported_tasks': list(SUPPORTED_TASKS.keys()),
    }

@app.post('/api/jules/quality')
async def api_quality(request: Request):
    data = await request.json()
    return analyze_code_quality(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/refactor')
async def api_refactor(request: Request):
    data = await request.json()
    return suggest_refactoring(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/security')
async def api_security(request: Request):
    data = await request.json()
    return security_scan(data.get('code', ''), data.get('language', 'javascript'))

@app.post('/api/jules/similarity')
async def api_similarity(request: Request):
    data = await request.json()
    return find_similar_code(data.get('blocks', []))

@app.post('/api/tasks/execute')
async def execute_task(request: Request):
    data = await request.json()
    task_type = data.get('type', 'code_quality')
    payload = data.get('payload', data)
    try:
        if task_type == 'code_quality':
            result = analyze_code_quality(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type == 'optimization' or task_type == 'refactor':
            result = suggest_refactoring(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type == 'security_scan':
            result = security_scan(payload.get('code', ''), payload.get('language', 'javascript'))
        elif task_type == 'similarity':
            result = find_similar_code(payload.get('blocks', []))
        else:
            result = analyze_code_quality(payload.get('code', ''), payload.get('language', 'javascript'))
        return {'success': True, 'node_id': HEADY_CONFIG['node_id'], 'task_type': task_type, 'result': result}
    except Exception as e:
        return {'success': False, 'error': str(e), 'node_id': HEADY_CONFIG['node_id']}

print('JULES FastAPI ready: /health, /api/jules/quality, /api/jules/refactor, /api/jules/security, /api/jules/similarity, /api/tasks/execute')

In [ ]:
# Cell 6: ngrok tunnel + auto-register with HeadyCloud
from pyngrok import ngrok, conf

# Authenticate ngrok
conf.get_default().auth_token = "39ZBirdUD63xgta7yN7OFZpE84m_3QZyJTDno1b8Yhv9Nfy8s"

public_url = ngrok.connect(HEADY_CONFIG['port']).public_url
print(f'JULES GPU Node live at: {public_url}')

PUBLIC_URL = public_url

async def register_with_clouds():
    async with httpx.AsyncClient(timeout=15) as client:
        for endpoint in HEADY_CONFIG['registration_endpoints']:
            try:
                resp = await client.post(endpoint, json={
                    'node_id': HEADY_CONFIG['node_id'],
                    'url': PUBLIC_URL,
                    'role': HEADY_CONFIG['node_role'],
                    'capabilities': HEADY_CONFIG['capabilities'],
                    'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                })
                print(f'  Registered with {endpoint}: {resp.status_code}')
            except Exception as e:
                print(f'  Pending: {endpoint} ({e})')

async def heartbeat_loop():
    while True:
        await asyncio.sleep(30)
        async with httpx.AsyncClient(timeout=10) as client:
            for layer, url in HEADY_CONFIG['cloud_layers'].items():
                try:
                    await client.post(f'{url}/api/nodes/heartbeat', json={
                        'node_id': HEADY_CONFIG['node_id'],
                        'status': 'active',
                        'url': PUBLIC_URL,
                        'metrics': {
                            'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'cpu',
                            'gpu_memory_used': round(torch.cuda.memory_allocated(0) / 1e9, 2) if torch.cuda.is_available() else 0,
                            'models_loaded': 2,
                        }
                    })
                except:
                    pass

asyncio.run(register_with_clouds())
threading.Thread(target=lambda: asyncio.run(heartbeat_loop()), daemon=True).start()
print(f'JULES GPU node live at {PUBLIC_URL} — accepting code analysis requests')
# 0.0.0.0 is the Colab VM bind address for uvicorn, NOT a local service
uvicorn.run(app, host='0.0.0.0', port=HEADY_CONFIG['port'])